### Setelah data diambil dari twitter dalam bentuk html, sekarang kita akan mengubah html tsb ke bentuk csv, html tiap bulan jadi csv

In [78]:
#bulan Oktober
fileSource = 'C:/WinPython_64bit/notebooks/analisis gopay/gopay OR go-pay since 2018-10-01 until 2018-10-31 - Twitter Search.html'
fileOutput = 'C:/WinPython_64bit/notebooks/analisis gopay/gopay OR go-pay since 2018-10-01 until 2018-10-31 - Twitter Search.csv'

TS.twitter_html2csv(fileSource, fileOutput)

Loading Data: 


189it [00:02, 82.07it/s]


Saving Data to "C:/WinPython_64bit/notebooks/analisis gopay/gopay OR go-pay since 2018-10-01 until 2018-10-31 - Twitter Search.csv" 
All Finished


In [79]:
#bulan November
fileSource = 'C:/WinPython_64bit/notebooks/analisis gopay/gopay OR go-pay since 2018-11-01 until 2018-11-30 - Twitter Search.html'
fileOutput = 'C:/WinPython_64bit/notebooks/analisis gopay/gopay OR go-pay since 2018-11-01 until 2018-11-30 - Twitter Search.csv'

TS.twitter_html2csv(fileSource, fileOutput)

Loading Data: 


208it [00:02, 83.29it/s]


Saving Data to "C:/WinPython_64bit/notebooks/analisis gopay/gopay OR go-pay since 2018-11-01 until 2018-11-30 - Twitter Search.csv" 
All Finished


In [80]:
#bulan Desember
fileSource = 'C:/WinPython_64bit/notebooks/analisis gopay/gopay OR go-pay since 2018-12-01 until 2018-12-31 - Twitter Search.html'
fileOutput = 'C:/WinPython_64bit/notebooks/analisis gopay/gopay OR go-pay since 2018-12-01 until 2018-12-31 - Twitter Search.csv'

TS.twitter_html2csv(fileSource, fileOutput)

Loading Data: 


280it [00:03, 83.45it/s]


Saving Data to "C:/WinPython_64bit/notebooks/analisis gopay/gopay OR go-pay since 2018-12-01 until 2018-12-31 - Twitter Search.csv" 
All Finished


### semua csv digabung jadi 1 csv

In [131]:
#gabungin semua csv

import os
import glob
import pandas

def concatenate(indir='C:/WinPython_64bit/notebooks/analisis gopay/perbulan/',outfile='C:/WinPython_64bit/notebooks/analisis gopay/perbulan/gabungan.csv'):
    os.chdir(indir)
    fileList=glob.glob("*.csv")
    dfList=[]
    colnames=["Time","Username","Tweet","Replies","Retweets","Likes","Language","urlStatus"]
    for filename in fileList:
        print(filename)
        df=pandas.read_csv(filename,header=None)
        dfList.append(df)
    concatDF=pandas.concat(dfList,axis=0)
    concatDF.columns=colnames
    concatDF.to_csv(outfile,index=None)

In [132]:
concatenate() #ini isi csv-nya ada apa aja

gopay OR go-pay since 2018-10-01 until 2018-10-31 - Twitter Search.csv
gopay OR go-pay since 2018-11-01 until 2018-11-30 - Twitter Search.csv
gopay OR go-pay since 2018-12-01 until 2018-12-31 - Twitter Search.csv


#membuat csv menjadi list
import pandas as pd

data = pd.read_csv('gabungan.csv')
Data = []
for i in range(len(data)):
    Data.append(data[' Tweet'][i])
len(Data) #panjang data

In [136]:
data.head() #diliatin beberapa data awal

,Time,Username,Tweet,Replies,Retweets,Likes,Language,urlStatus
0,30-Oct-18,@gagalselingkuh,bismillah...\r\n\r\nmau giveaway Rp 50.000 dal...,613,1037,222,Indonesian,https://twitter.com/gagalselingkuh/status/1057...
1,30-Oct-18,@askmenfess,[askmf] siapa yg mau saldo gopay? https://twit...,7,2,1,Indonesian,https://twitter.com/askmenfess/status/10572635...
2,30-Oct-18,@muoonlight,GA gopay & pulsa all operator masing2 10k. \r\...,40,541,50,Indonesian,https://twitter.com/muoonlight/status/10572458...
3,30-Oct-18,@AsikinHidup,"Sambil nungguin gue topup gopay, gimana kalo g...",43,23,85,Indonesian,https://twitter.com/AsikinHidup/status/1057234...
4,30-Oct-18,@dreamyoonjh,[GIVEAWAY]\r\nKarena hari ini aku ultah dan di...,22,52,43,Indonesian,https://twitter.com/dreamyoonjh/status/1057212...


### mulai preprocessing

In [13]:
from bs4 import BeautifulSoup as bs
import re,csv, os, itertools
from tqdm import tqdm
from nltk.tokenize import RegexpTokenizer
from nltk.stem import WordNetLemmatizer
from nltk import sent_tokenize, word_tokenize
from spacy.lang.id import Indonesian
from html import unescape
from unidecode import unidecode


fData = 'gabungan.csv'
fName="gabungan_bersih.csv"
fSlang = 'C:/WinPython_64bit/notebooks/analisis gopay/perbulan/slang.dic'
dPath = 'C:/WinPython_64bit/notebooks/analisis gopay/perbulan/'
bahasa = 'id'

#ngilangin stopword

def LoadStopWords(lang):
    L = lang.lower().strip()
    if L == 'en' or L == 'english' or L == 'inggris':
        lemmatizer = WordNetLemmatizer()
        stops =  set([t.strip() for t in LoadDocuments(file = 'C:/WinPython_64bit/notebooks/analisis gopay/perbulan/stopwords_eng.txt')[0]])
    elif L == 'id' or L == 'indonesia' or L=='indonesian':
        lemmatizer = Indonesian() 
        stops = set([t.strip() for t in LoadDocuments(file = 'C:/WinPython_64bit/notebooks/analisis gopay/perbulan/stopwords_id.txt')[0]])
    else:
        print('Warning, language not recognized. Empty StopWords Given')
        stops = set(); lemmatizer = None
    return stops, lemmatizer

# HashTags Processing belum ada
def crawlFiles(dPath,types=None): # dPath ='C:/Temp/', types = 'pdf'
    if types:
        return [dPath+'/'+f for f in os.listdir(dPath) if f.endswith('.'+types)]
    else:
        return [dPath+'/'+f for f in os.listdir(dPath)]
    
def fixTags(T):
    getHashtags = re.compile(r"#(\w+)")
    pisahtags = re.compile(r'[A-Z][^A-Z]*')
    t = T
    tagS = re.findall(getHashtags, T)
    for tag in tagS:
        proper_words = ' '.join(re.findall(pisahtags, tag))
        t = t.replace('#'+tag,proper_words)
    return t

def LoadDocuments(dPath=None,types=None, file = None): # types = ['pdf','doc','docx','txt','bz2']
    Files, Docs = [], []
    if types:
        for tipe in types:
            Files += crawlFiles(dPath,tipe)
    if file:
        Files = [file]
    if not types and not file: # get all files regardless of their extensions
        Files += crawlFiles(dPath)
    for f in Files:
        if f[-3:].lower()=='txt':
            try:
                df=open(f,"r",encoding="utf-8", errors='replace')
                Docs.append(df.readlines());df.close()
            except:
                print('error reading{0}'.format(f))
        else:
            print('Unsupported format {0}'.format(f))
    if file:
        Docs = Docs[0]
    return Docs, Files

# filtering

# T sembarang string (e.g. kalimat)
def cleanText(T, fix={}, lang = 'en', lemmatizer=None, stops = set(), symbols_remove = False, min_charLen = 0): 
    # lang & stopS only 2 options : 'en' atau 'id'
    # symbols ASCII atau alnum
    pattern = re.compile(r'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+')
    t = re.sub(pattern,' ',T) #remove urls if any
    t = unescape(t) # html entities fix
    t = fixTags(t) # fix abcDef
    t = t.lower().strip() # lowercase
    t = unidecode(t)
    t = ''.join(''.join(s)[:2] for _, s in itertools.groupby(t)) # remove repetition
    t = sent_tokenize(t) # sentence segmentation. String to list
    for i, K in enumerate(t):
        if symbols_remove:
            K = re.sub(r'[^.,a-zA-Z0-9 \n\.]',' ',K)
        
        cleanList = []
        if lang =='en':
            listKata = word_tokenize(K) # word tokenize
            for token in listKata:
                if token in fix.keys():
                    token = fix[token]
                if lemmatizer:
                    token = lemmatizer.lemmatize(token)
                if stops:
                    if len(token)>=min_charLen and token not in stops:
                        cleanList.append(token)
                else:
                    if len(token)>=min_charLen:
                        cleanList.append(token)
            t[i] = ' '.join(cleanList)
        else:
            K = lemmatizer(K)
            listKata = [token.text for token in K]
            for token in listKata:
                if token in fix.keys():
                    token = fix[token]
                
                if lemmatizer:
                    token = lemmatizer(token)[0].lemma_
                if stops:    
                    if len(token)>=min_charLen and token not in stops:
                        cleanList.append(token)
                else:
                    if len(token)>=min_charLen:
                        cleanList.append(token)
            t[i] = ' '.join(cleanList)
    return ' '.join(t) # Return kalimat lagi

if __name__ == "__main__":
    print('Loading StopWords & Slang ... ', flush = True)
    stops, lemmatizer = LoadStopWords(bahasa)
    sw=open(fSlang,encoding='utf-8', errors ='ignore', mode='r');SlangS=sw.readlines();sw.close()
    SlangS = {slang.strip().split(':')[0]:slang.strip().split(':')[1] for slang in SlangS}
    # Dictionary comprehension, bentuk umum:
    # D = {k:v for v in List}
    tokenizer=RegexpTokenizer(r'\w+')
    
    print('Loading Data: ', flush = True)
    allData = crawlFiles(dPath, types='html')
    Tweets, Clean_Tweets, Username, waktu, replies, retweets, likes, Language = [], [], [], [], [], [], [], []
    # fData = allData[0]
    for fData in tqdm(allData):
        soup = bs(open(fData,encoding='utf-8', errors = 'ignore', mode='r'),'html.parser')
        data = soup.find_all('li', class_= 'stream-item')
        # t = data[0]
        for t in data:
            try:
                # Loading tweet
                T = t.find_all('p',class_='TweetTextSize')[0]
                T = bs(str(T),'html.parser').text
                Tweets.append(T)
                T = cleanText(T, fix=SlangS, lang = bahasa, lemmatizer=lemmatizer, stops = stops, symbols_remove = True, min_charLen = 2)
                Clean_Tweets.append(T)
                # Loading UserName
                U = t.find_all('span',class_='username')
                U = bs(str(U[0]),'html.parser').text
                Username.append(U)
                # Loading Time
                T = t.find_all('a',class_='tweet-timestamp')[0]
                T = bs(str(T),'html.parser').text
                waktu.append(T)
                # Loading reply, retweet & Likes
                RP = t.find_all('span',class_='ProfileTweet-actionCountForAria')[0]
                RT = t.find_all('span',class_='ProfileTweet-actionCountForAria')[1]
                L  = t.find_all('span',class_='ProfileTweet-actionCountForAria')[2] 
                RP = int(bs(str(RP), "lxml").text.split()[0])
                RT = int(bs(str(RT), "lxml").text.split()[0])
                L = int(bs(str(L), "lxml").text.split()[0])
                replies.append(RP)
                retweets.append(RT)
                likes.append(L)
                # Loading Bahasa
                try:
                    L = t.find_all('span',class_='tweet-language')
                    L = bs(str(L[0]), "lxml").text
                except:
                    L =''
                Language.append(L)
            except:
                pass

    print('Saving Data to "%s" ' %fName, flush = True)
    N = len (Tweets)
    dfile = open(fName, 'w', encoding='utf-8', newline='')
    dfile.write('Username,Time,Tweet,Cleaned_Tweet\n')
    # Tweets, Clean_Tweets, Username, waktu, replies, retweets, likes, Language
    with dfile :
        writer = csv.writer(dfile)
        for i in range(N):
            writer.writerow([Username[i],waktu[i],Tweets[i],Clean_Tweets[i]])
    dfile.close()
             
    print('All Finished', flush = True)

Loading StopWords & Slang ... 
Loading Data: 


100%|████████████████████████████████████████████████████████████████████████████████████| 3/3 [00:21<00:00,  7.18s/it]


Saving Data to "gabungan_bersih.csv" 
All Finished


In [3]:
fData = 'gabungan.csv'
fName="gabungan_bersih.csv"
fSlang = 'C:/WinPython_64bit/notebooks/analisis gopay/slang.dic'
dPath = 'C:/WinPython_64bit/notebooks/analisis gopay/perbulan'

### tweet belum dibersihkan

In [3]:
import pandas as pd

data = pd.read_csv('C:/WinPython_64bit/notebooks/analisis gopay/perbulan/gabungan.csv')
data.head()
Data = data[' Tweet']
Data[:5]

0    bismillah...\r\n\r\nmau giveaway Rp 50.000 dal...
1    [askmf] siapa yg mau saldo gopay? https://twit...
2    GA gopay & pulsa all operator masing2 10k. \r\...
3    Sambil nungguin gue topup gopay, gimana kalo g...
4    [GIVEAWAY]\r\nKarena hari ini aku ultah dan di...
Name:  Tweet, dtype: object

### tweet sudah dibersihkan

In [143]:
import pandas as pd

df = pd.read_csv('C:/WinPython_64bit/notebooks/analisis gopay/perbulan/gabungan_bersih.csv')
df.head()
Tweets = df['Cleaned_Tweet']
Tweets[:5]

0    bismillah .. \n\n rp 50.00 bentuk    pulsa beb...
1                                       askmf saldo ..
2       pulsa all operator masing2 10k rulesnya rt ...
3               nungguin gue topup gimana terakir hari
4    hari ultah diumumkan superstar pledis saldo pa...
Name: Cleaned_Tweet, dtype: object

In [159]:
import os  
os.path.isfile('C:/WinPython_64bit/notebooks/analisis gopay/perbulan/gabungan_bersih _labeled.csv')

True

#### ini kalo coba diliatin hasil2 dari masing2 tahap preprocessing, namun komputasinya mungkin lama, maka biasanya preprocessing itu prosesnya langsung semua

In [4]:
print(Data[10])

Nah! Di app GO-JEK sekarang, kita bisa #KasihLebih juga pake GO-PAY! Bentuk apresiasi buat orang yang mageran kaya gue. Dibantu kirim2 barang, dan gue bisa berterima kasih dalam bentuk nyata. https://twitter.com/gojekindonesia/status/1053518908777410560 …


In [7]:
#menghilangkan url
import re
urlPattern = re.compile(r'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+')

#txt = 'website reguler expression & my site : https://www.regular-expressions.info/ & http://taufiksutanto.blogspot.com'
filter=re.sub(urlPattern,' ',Data[10])
print(filter)


Nah! Di app GO-JEK sekarang, kita bisa #KasihLebih juga pake GO-PAY! Bentuk apresiasi buat orang yang mageran kaya gue. Dibantu kirim2 barang, dan gue bisa berterima kasih dalam bentuk nyata.   …


In [12]:
# filtering simbol

import re
filtering=re.sub(r'[^\w]',' ',filter)
print(filtering)
# atau jika ingin exclude titik dan koma 
# re.sub(r'[^.,a-zA-Z0-9 \n\.]','',txt)

Nah  Di app GO JEK sekarang  kita bisa  KasihLebih juga pake GO PAY  Bentuk apresiasi buat orang yang mageran kaya gue  Dibantu kirim2 barang  dan gue bisa berterima kasih dalam bentuk nyata     


In [13]:
# case folding = nyamain semua huruf jadi kecil

casefolding=filtering.lower() #save hasil case folding dgn varibel baru, utk nanti dipanggil di langkah selanjutnya
print(casefolding) # x[10] -> variabel x di baris ke-10

nah  di app go jek sekarang  kita bisa  kasihlebih juga pake go pay  bentuk apresiasi buat orang yang mageran kaya gue  dibantu kirim2 barang  dan gue bisa berterima kasih dalam bentuk nyata     


In [21]:
#hapus stopword

import TSutanto_lib as ittc
from nltk.corpus import stopwords
from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory
factory = StopWordRemoverFactory()
NLTK_StopWords = stopwords.words('english')
astrawi_StopWords_id = factory.get_stop_words()
Personal_StopWords_en = [t.strip() for t in ittc.LoadDocuments(file = 'data/stopwords_eng.txt')[0]]
Personal_StopWords_id = [t.strip() for t in ittc.LoadDocuments(file = 'data/stopwords_id.txt')[0]]
Sastrawi_StopWords_id = factory.get_stop_words()
NLTK_StopWords = set(NLTK_StopWords)
Sastrawi_StopWords_id = set(Sastrawi_StopWords_id)
Personal_StopWords_en = set(Personal_StopWords_en)
Personal_StopWords_id = set(Personal_StopWords_id)

T = casefolding
T_en = [t for t in word_tokenize(T) if t not in NLTK_StopWords]
print(' '.join(T_en))
T_id = [t for t in word_tokenize(T) if t not in Sastrawi_StopWords_id]
print(' '.join(T_id))

error readingdata/stopwords_eng.txt


IndexError: list index out of range

In [15]:
# Spacy Lemmatizer menggunakan entitas wordnet (akan dibahas lebih detail di segmen berikutnya)
#hapus stopword

import spacy
from spacy.lang.id import Indonesian
nlp_en = spacy.load('en')
nlp_id = Indonesian() 

idn = nlp_id(casefolding)
print( ' '.join( k.lemma_ for k in idn ) )

nah   di app go jek sekarang   kita bisa   kasihlebih juga pake go pay   bentuk apresiasi buat orang yang mageran kaya gue   dibantu kirim2 barang   dan gue bisa terima kasih dalam bentuk nyata     


In [11]:
# tokenisasi = misahin kata per kata

from nltk.tokenize import TweetTokenizer
Tokenizer = TweetTokenizer(strip_handles=True, reduce_len=True)
tokenisasi=Tokenizer.tokenize(filtering) # tokenisasi dari hasil filtering di atas
print(tokenisasi)

['nah', 'di', 'app', 'go', 'jek', 'sekarang', 'kita', 'bisa', 'kasihlebih', 'juga', 'pake', 'go', 'pay', 'bentuk', 'apresiasi', 'buat', 'orang', 'yang', 'mageran', 'kaya', 'gue', 'dibantu', 'kirim', '2', 'barang', 'dan', 'gue', 'bisa', 'berterima', 'kasih', 'dalam', 'bentuk', 'nyata']


In [204]:
#slang

Slangs = TS.LoadDocuments(file = 'C:/WinPython_64bit/notebooks/data/slang.dic')[0]
    
SlangDict={}
for slang in Slangs:
    try:
        key, value = slang.split(':')
        SlangDict[key.strip()]= value.strip()
    except:
        pass

In [ ]:
import pickle
from tqdm import tqdm_notebook as tqdm
 
stops, lemmatizer = TS.LoadStopWords(lang='id')
for i,d in tqdm(enumerate(Tweets)):
    Tweets[i] = TS.cleanText(d, SlangDict, lemma=lemmatizer, stops=None, symbols_remove=True, min_charLen=2)
print(Tweets[:5])

In [149]:
# hapus stopword, belom tau caranya

### tweet yang sudah diberi label

### membentuk VSM

In [195]:
#Membentuk VSM
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(ngram_range=(2, 2)) #ngram itu feature selection
vsm = vectorizer.fit_transform(Tweets)
vsm = vsm[vsm.getnnz(1)>0][:,vsm.getnnz(0)>0]
vsm.shape # bentuk vsm dari kolom 'tweet' csv bersih

(677, 10244)

In [196]:
print(vsm) # di baris 0 kolom 1821 nilai katanya (tfidf) 0.21 dst

  (0, 1821)	0.21869641997484815
  (0, 7891)	0.19648885998078694
  (0, 446)	0.16129071015768456
  (0, 7)	0.21869641997484815
  (0, 1533)	0.17855947711269207
  (0, 7513)	0.21869641997484815
  (0, 1393)	0.1834982701517458
  (0, 7254)	0.21869641997484815
  (0, 2907)	0.21869641997484815
  (0, 5767)	0.205705830145807
  (0, 10026)	0.21869641997484815
  (0, 5611)	0.21869641997484815
  (0, 476)	0.17428129998672573
  (0, 7677)	0.21869641997484815
  (0, 7798)	0.17855947711269207
  (0, 704)	0.21869641997484815
  (0, 7769)	0.21869641997484815
  (0, 4105)	0.21869641997484815
  (0, 2204)	0.21869641997484815
  (0, 2205)	0.21869641997484815
  (0, 2475)	0.21869641997484815
  (0, 6419)	0.21869641997484815
  (0, 10045)	0.21869641997484815
  (1, 1004)	1.0
  (2, 7512)	0.2983827579912732
  :	:
  (676, 8562)	0.16532033125332815
  (676, 9247)	0.16532033125332815
  (676, 971)	0.3306406625066563
  (676, 8161)	0.16532033125332815
  (676, 2941)	0.16532033125332815
  (676, 9038)	0.16532033125332815
  (676, 236)	0.1

In [197]:
import pandas as pd

datalabel = pd.read_csv('C:/WinPython_64bit/notebooks/analisis gopay/perbulan/gabungan_bersih _labeled.csv')
datalabel.head()
label = datalabel['Label']
label[:5]

0    0
1    0
2    1
3    0
4    0
Name: Label, dtype: int64

In [198]:
# Split data menjadi data train dan test

from sklearn.cross_validation import train_test_split
X = datalabel.Cleaned_Tweet #dari csv gabungan_bersih yang udah dilabelin aja
y = datalabel.Label
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1, test_size=0.3)
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)


(330,)
(142,)
(330,)
(142,)


### membentuk model

## 1. SVM

In [199]:
from sklearn import svm
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.model_selection import cross_val_score
import time

dSVM = svm.SVC(decision_function_shape='ovo') #sudah dicoba 'sigmoid', 'poly' dan 'rbf' dan menghasilkan hal yang tidak jauh berbeda
mulai = time.time()
#melakukan Cross-Validation dengan SVM
scores_svm = cross_val_score(dSVM, X_train, y_train, cv=10,scoring='accuracy') # perhatikan sekarang kita menggunakan seluruh data
waktu = time.time() - mulai
# Interval Akurasi 95 CI 
print("Accuracy SVM: %0.2f (+/- %0.2f), Waktu = %0.3f detik" % (scores_svm.mean(), scores_svm.std() * 2, waktu))

ValueError: could not convert string to float: 'plis ovo sm saing sobat missqueen ku nikmat indahnya hidup'

In [169]:
# Fitting and evaluate the model
dSVM = svm.SVC(C = 10**5)
dSVM.fit(X_train, y_train)
y_SVM = dSVM.predict(X_test)
print('Akurasi = ', accuracy_score(y_test, y_SVM))
print(confusion_matrix(y_test, y_SVM))
print(classification_report(y_test, y_SVM))

ValueError: could not convert string to float: 'nctsmtown 127 regular4th win moment jaeyong taeyong mbak boa \n\n ga total pulsa masing2 25k orang menang \n\n rt aja'

In [177]:
from sklearn import svm
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.model_selection import cross_val_score
import time

dSVM = svm.SVC(decision_function_shape='ovo') #sudah dicoba 'sigmoid', 'poly' dan 'rbf' dan menghasilkan hal yang tidak jauh berbeda
dSVM.fit(X_train,y_train)
y_SVM=dSVM.predict(X_test)
accuracy_score(y_test,y_SVM)


ValueError: could not convert string to float: 'nctsmtown 127 regular4th win moment jaeyong taeyong mbak boa \n\n ga total pulsa masing2 25k orang menang \n\n rt aja'